In [6]:
import imageio
import numpy as np
from math import exp

#sigmoid function
def sig(s):
    if s < 0:
        return 1 - 1 / (1 + exp(s))
    return 1 / (1 + exp(-s))

def train(updatex,updatey):
    #process x
    a = np.ones(updatex.shape[0])
    updatex = np.insert(updatex, 0, values=a, axis=1) #184*961
    updatex = updatex/255.0
    #random set w and w2
    update_w = np.random.randint(-1, 2, size=[961, 100])  # 961*100
    update_w2 = np.random.randint(-1, 2, size=[101, 1])  # 101*1
#     datanupw = -0.01+0.02*np.random.random((961, 100))  # 961*100
#     datanupw2 = -0.01+0.02*np.random.random((101, 1))  # 101*1
    
    for i in range(0, 1000):
        #forward direction
        #random pickup t 
        t = np.random.randint(0, updatex.shape[0], 1)
        #level l-1 :randomx, updatew, update_s
        randomx = updatex[t]  # 1*961 
        y = updatey[t]
        y = float(y)
        update_s = (randomx * update_w).tolist() #1*100 #s=x*w的和
        #level l-1:sigmoid
        s_l=[]
        for j in range(len(update_s[0])):
            l=[]
            l.append(sig(update_s[0][j]))
            s_l.append(l) 
        updates=np.array(s_l).T#1*100 

        b = np.ones(1)
        #level l: updatex2, updatew2, update_s2
        updatex2 = np.insert(updates, 0, values=b, axis=1)#1*101
        update_s2 = updatex2.dot(update_w2) #1*1
        #level l:sigmoid
        updatex3 = sig(update_s2) #updatex3: output of the output layer 
        
        #back dorection
        #level l: g1
        g1 = 2.0 *(updatex3 - y)* updatex3 * (1.0 - updatex3)
        #level l:update w2=w2-learning_rate*g1*x1.T
        w2 = update_w2 - 0.1 * g1 * updatex2.T #101*1 
        
        #level l-1: g2 = np.dot((datanupx2.T,(1.0 - datanupx2)), np.dot(w2,g1)) #101*1 ---
        a = updatex2 * (1.0 - updatex2)#1*101*1*101, multiply not dot
        g2 =np.matrix(a.T * np.dot(w2,g1)) #101*1
        g2 = np.delete(g2,0,0) #100*1
        #level l-1: updatew=w-(g2*x*0.1).T
        w = update_w - np.dot(0.1,g2 * randomx).T#961*100
        update_w2 = w2
        update_w = w
    return w,w2

def test(updatex,updatey,w,w2):
    count = 0
    prediction = []
    lable=updatey
    a = np.ones(updatex.shape[0])
    updatex = np.insert(updatex, 0, values=a, axis=1)  # 83*961
    updatex = updatex / 255.0 
    for i in range(0, updatey.shape[0]):
        x = updatex[i,:] #1*961
        y = updatey[i,0]
        y = float(y)
        #datanup_s = np.dot(siglex,w) # 1*100
        update_s =np.dot(x,w).tolist() #1*100 #s=x*w的和
        s_l=[]
        for j in range(len(update_s[0])):
            l=[]
            l.append(sig(update_s[0][j]))
            s_l.append(l) 
        updates=np.array(s_l).T #1*100 
       
        b = np.ones(1)
        updatex2 = np.insert(updates, 0, values=b, axis=1) # 1*101
        update_s2 = updatex2.dot(w2)  # 1*1
        updatex3 = sig(update_s2)
        prediction.append(updatex3)
   
    #compare prediction and the label
    for j in range(len(prediction)):
        if prediction[j] >= 0.5:
            prediction[j] = 1.0
        else:
            prediction[j] = 0.0
        if prediction[j] == lable[j]:
            count += 1.0
    accuracy = count/updatex.shape[0]
    print ("prediction_list = ", prediction)
    print ("accuracy = ", accuracy)

#get training data
f = open("downgesture_train.list.txt")
train_x = []
train_y = []
for line in f.readlines():
    line = line.strip()
    if "down" in line:
        train_y.append(1.0)
    else:
        train_y.append(0.0)
    c = imageio.imread(line)
    m = c.shape[0]*c.shape[1]
    x = list(c.reshape(1,m))
    train_x += x
trainx = np.matrix(train_x) #184*960
trainy = (np.matrix(train_y)).T #184*1
w,w2 = train(trainx,trainy)
#get testing data
f2 = open("downgesture_test.list.txt")
test_x = []
test_y = []
for line in f2.readlines():
    line = line.strip()
    if "down" in line:
        test_y.append(1)
    else:
        test_y.append(0)
    c2 = imageio.imread(line)
    m2 = c2.shape[0]*c2.shape[1]
    x2 = list(c2.reshape(1,m2))
    test_x += x2
testx = np.matrix(test_x) #83*960
testy = (np.matrix(test_y)).T #83*1
test(testx,testy,w,w2)

prediction_list =  [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
accuracy =  0.8433734939759037
